# Alternative Vorgehensweise: Data2CSV und Datenbereinigung

## Vorgehen

1. Webscraping (HTML in pre-Tag)
2. b-Tags (und andere Tags) löschen d.h. alles zwischen Spitzklammern <>
3. Aufsplitten des Texts in Textblöcke anhand von Leerzeilen (2x \n nacheinander), Textblöcke als Array speichern
4. Textblöcke kategorisieren, typische Merkmale identifizieren, Verarbeitungsanweisungen für die einzelnen Textblöcke definieren
5. in CSV umwandeln, dabei die Verarbeitungsanweisungen als if-Bedingungen umsetzen (jeden Textblocktyp einzeln verarbeiten)
6. weitere automatisierte Datenbereinigungsschritte: Fragezeichen ersetzen, bestimmte Textteile löschen z.B. (V.O.) etc.
7. händische Datenbereinigung (restliche Fragezeichen ersetzen, Regieanweisungen die direkt am Sprechertext kleben ausfindig machen und in eine eigene Regie-Zeile auslagern
8. Zusammenführen der 3 CSV-Dateien in eine (zusätzliche Spalte für Film-Nummer)

Beispiel für Textblöcke:

**1.) Textblock vom Typ "Sprechertext"**

>**GALADRIEL (V.O.) (CONT'D)**

          But the power of the Ring could not be
          undone.
          

typisches Merkmal: beginnt mittig (x Leerzeichen Abstand vom Rand, genaue Anzahl der Leerzeichen variiert je nach Filmskritpt!)

Verarbeitung: Text in erster Zeile (Name des Sprechers) wird in linke Spalte gepackt, Text ab zweiter Zeile wird in rechte Spalte gepackt

**2.) Textblock vom Typ "Regieanweisung"** (Skript 1 und 3)

>IMAGES: THE HUGE, DARK FIGURE OF SARURON, bearing the ONE
RING on his finger, looms over the field of battle...

typisches Merkmal: beginnt linksbündig (! ausgenommen Textblöcke, in denen nur **CONTINUED:** steht)

Verarbeitung: in linke Spalte wird "Regie" geschrieben, Text kommt in rechte Spalte, (Wörter vorm Doppelpunkt werden später gelöscht)

**3.) Textblock vom Typ "Regieanweisung"** (Skript 2)

>[The hall is shown to be filled with 
                         light again, as everyone marvels at 
                         the rejuvenation of the king.]

typisches Merkmal: steht in []

Verarbeitung: wie 2. (linke Spalte: "Regie", rechte Spalte: Text ohne Klammern)

**Vorteil: alle Textteile, für die keine Verarbeitungsanweisung definiert wird, werden auch nicht mit ins CSV aufgenommen.** Man zieht sich nur die Teile raus, die man braucht. z.B. werden Seitenzahlen nicht berücksichtigt, weil sie rechtsbündig stehen

## Ziel: CSV-Datei in folgendem Format

| Nr. | Regie/Sprecher | Text | Film-Nr. |


| 1 | Gollum | My precious! | 1 |

| 2 | Regie | Gollum looks at the ring. | 1 |

### Skript 1 und 3

In [8]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [9]:
lotr_1 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-Fellowship-of-the-Ring,-The.html')
#lotr_2 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-The-Two-Towers.html')
lotr_3 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-Return-of-the-King.html')

#scripts = [lotr_1, lotr_2, lotr_3]
scripts1_3 = [lotr_1, lotr_3]

In [8]:
def get_scripts():
    # Schleife für jedes Script
    for movie, script in enumerate(scripts1_3, 1):
        # webscrape text
        soup = BeautifulSoup(script.text, 'lxml')
        all_character_tags = soup.find('pre')
        
        # convert scraped bs4 element tags to string
        text = str(all_character_tags)
        
        # delete all tags
        text = re.sub(r'</?[a-zA-Z]+>', '', text)
        
        # split to text blocs (Trenner: Leerzeilen = 2x new line = \r\n\r\n)
        blocs = re.split(r'\r\n\r\n', text)
        
        # print(blocs)
        for num, bloc in enumerate(blocs):
            # matches Regieanweisungen, d.h. alle Blöcke, die direkt mit Buchstaben beginnen
            #if(re.match(r'^\s*\[', bloc)):
            # if(re.match(r'^\[?[a-zA-Z]', bloc)):
                print(num, bloc)
                
get_scripts()

0 BLACK SCREEN
1 SUPER: New Line Cinema Presents
2 SUPER: A Wingnut Films Production
3 BLACK CONTINUES... ELVISH SINGING....A WOMAN'S VOICE IS
whispering, tinged with SADNESS and REGRET:
4                     GALADRIEL (V.O.)
              (Elvish: subtitled)
          "I amar prestar sen: han mathon ne nen,
          han mathon ne chae...a han noston ned
          wilith."
              (English:)
          The world is changed: I feel it in the
          water, I feel it in the earth, I smell it
          in the air...Much that once was is lost,
          for none now live who remember it.
5 SUPER: THE LORD OF THE RINGS
6 EXT. PROLOGUE -- DAY
7 IMAGE: FLICKERING FIRELIGHT. The NOLDORIN FORGE in EREGION.
MOLTEN GOLD POURS from the lip of an IRON LADLE.
8                     GALADRIEL (V.O.)
          It began with the forging of the Great
          Rings.
9 IMAGE: THREE RINGS, each set with a single GEM, are received
by the HIGH ELVES-GALADRIEL, GIL-GALAD and CIRDAN.
10               

1454 
                    ARAGORN
              (ominous)
          Do not disturb the water.
1455 Aragorn watches anxiously as the Ripples appear to grow....he
exchanges a look with Boromir.
1456 Aragorn's hand creeps towards his sword. Gandalf gives up in
despair...he sits down beside Frodo. Close on: Frodo peers at
the Elvish inscription...his face breaks into a smile of
comprehension.
1457                     FRODO
              (quietly)
          It's a riddle...
1458 Gandalf raises his eyebrows...
1459                     FRODO (CONT'D)
              (explaining)
          Speak, friend, and enter.   What's the
          Elvish for friend?
1460                     GANDALF
          Oh...mellon.
1461 With that, the rock face silently divides in the middle and
two great Doors swing outwards... revealing a blackness
deeper than the night. As the Fellowship enter the Blackness,
something in the water stirs....
1462 INT. MORIA GATE -- NIGHT
1463 The Fellowship step warily into the da

obediently behind.
464 At the FAR END, upon a DAIS OF MANY STEPS, sits a LARGE,
EMPTY THRONE . . . and on the wall behind is the IMAGE of a
FLOWERING WHITE TREE set in GEMS.
465 At the FOOT of the DAIS, in a small STONE CHAIR, sits an OLD
MAN, gazing at his lap: DENETHOR - the Lord Steward of
CONDOR.
466                         GANDALF
             Hail, Denethor, Son of Ecthelion, Lord and
             Steward of Gondor.
467 DENETHOR does not look up as GANDALF and PIPPIN approach.
468                         GANDALF (cont'd)
             I come with tidings in this dark hour - and
             with counsel.
469 DENETHOR slowly raises COLD EYES . . .
470                          DENETHOR
                  (bitter)
             Perhaps you come to explain this?
471                                                            (CONTINUED)
472                                            Final Revision - October, 2003 31.
CONTINUED:
473 
DENETHOR holds up BOROMIR'S BROKEN HORN ...
474        

. . . A RED LIGHT seems to sweep across the PLAIN ...
1928                       FRODO (cont'd)
               (terror)
           It's him - the Eye.
                                                      (CONTINUED)
1929                                      Final Revision - October, 2003 121.
CONTINUED:
1930 
The GREAT EYE OF SAURON!
1931 SAM cannot see what FRODO sees. GENTLY, he urges FRODO on.
1932                        SAM
             We have to go in there, Mr Frodo. There's
             nothing for it. Come on. Let's just make it
             down the hill for starters.
1933 EXT. MINAS TIRITH, TOWER HALL - DAY
1934 CLOSE ON: GANDALF stands in the TOWER HALL . . . seemingly
alone . . . he speaks quietly, doubt and fear edge his voice.
1935                         GANDALF
             Frodo has passed beyond my sight. The
             darkness is deepening.
1936 He turns and crosses the HALL to reveal the presence of
GIMLI, LEGOLAS, EOMER and ARAGORN . . .
1937 ARAGORN, his back

### Skript 2

In [14]:
#lotr_1 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-Fellowship-of-the-Ring,-The.html')
lotr_2 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-The-Two-Towers.html')
#lotr_3 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-Return-of-the-King.html')

scripts_2 = [lotr_2]

In [16]:
def get_scripts():
    # Schleife für jedes Script
    for movie, script in enumerate(scripts_2, 1):
        # webscrape text
        soup = BeautifulSoup(script.text, 'lxml')
        all_character_tags = soup.find('pre')
        
        # convert scraped bs4 element tags to string
        text= str(all_character_tags)
        
        # delete all tags
        text= re.sub(r'</?[a-zA-Z]+>', '', text)
        
        # split intoTextblocs 
        # Problem: mehrere Trenner - \r\n\r\n und \r\n \r\n)
        blocs = re.split(r'(\r\n\s?\r\n)', text)
        
        # delete empty blocs (does not work yet!!!!)
        # nice to have, but not necessarily needed
        # blocs = [i for i in blocs if i != '\r\n \r\n' or i != '\r\n\r\n']

        # print(blocs)
        
        # PRINT ALL BLOCS
        #for num, bloc in enumerate(blocs):
         #   print(num, bloc)
        
        # PRINT ONLY REGIEANWEISUNGEN
        #counter = 0
        #for num, bloc in enumerate(blocs):
            # matches Regieanweisungen, d.h. alle Blöcke, die direkt mit [ beginnen
            #if(re.match(r'^\s*\[', bloc)):
               # counter += 1
               # print(num, bloc)
        #print(f'Anzahl der Regieanweisungen: {counter}')
        # Ergenbis: 363 (ohne die Regieanweisungen, die direkt hinter demTextstehen!)
        # Gesamtzahl an Regieanweisungen: 671 (Ergebnis der Suche nach [ in Skript 2)
        
        # PRINT ONLY SPRECHERTEXT
        #counter = 0
        #for num, bloc in enumerate(blocs):
           #  matches Sprechertext, d.h. alle Blöcke, die nicht mit [ beginnen
            #if(not re.match(r'^\s*\[', bloc)):
               # counter += 1
                # Sprechername ausgeben
                #if (re.match(r'^.*(?=(\r\n))', bloc)):
                   # name = re.match(r'^.*(?=(\r\n))', bloc).group(0)
                   # print(f'Name: {name}')
                # Sprechertext ausgeben
               # print(num, bloc)
                
       # print(f'Anzahl der Sprechertexte: {counter}')
        
        # WRITE TO CSV
        
        # Dateiname festlegen
        filename = f'lotr_skript2_test.csv'

        # Datei löschen, falls sie existiert:
        if os.path.exists(f'data/{filename}'):
            os.remove(f'data/{filename}')
            
            
        with open(f'data/{filename}', 'a') as f:
            # Spaltenüberschriften
            f.write(f'Nr.|Sprecher/Regie|Text|Filmnr.\n')
            print(f'Nr.|Sprecher/Regie|Text|Filmnr.\n')
                
            # Blöcke durchgehen
            for num, bloc in enumerate(blocs):
                # Leerzeichen entfernen und Fragezeichen ersetzen (erzeugt beim Ausführen sonst Unicode-Error)
                bloc = bloc.replace("�", "%")   
                
                # match Regieanweisungen d.h. alle Blöcke, die direkt mit [ beginnen
                if(re.match(r'^\s*\[', bloc)):
                    # remove whitespace
                    regieText = bloc.strip()
                    regieText = re.sub(r'\r\n', '', regieText) # Zeilenumbrüche innerhalb des Texts
                    regieText = re.sub(r'\s+', ' ', regieText) # mehrfache Leerzeichen werden zu einem
                    
                    # write to CSV
                    # Bsp: 1 | Regie | Frodo jumps. |                  
                    f.write(f'{num}|Regie|{regieText}|2\n')
                    print(f'{num}|Regie|{regieText}|2\n')
                    
                # match Sprechertext, d.h. alle Blöcke, die NICHT direkt mit [ beginnen
                elif(not re.match(r'^\s*\[', bloc)):
                    # überflüssiger whitespace vor dem Namen entfernen
                    # positives Lookahead, matched jegliche Art whitespace vor dem ersten Buchstaben
                    sprecherText = re.sub(r'^\s*(?=([a-zA-Z]))', "", bloc)
                    
                    # matched alles vor dem ersten Zeilenumbruch, d.h. den Sprechernamen
                    if (re.match(r'^.*(?=(\r\n))', sprecherText)):
                        # Name ohne Whitespace aus demTextherausziehen
                        # group(0) wird benötigt, um auf denTextim Match-Objekt zuzugreifen, das von re.match zurückgegeben wird
                        name = re.match(r'^.*(?=(\r\n))', sprecherText).group(0).strip()
                        
                        # alles vor dem ersten Zeilenumbruch (inkl. Name wird gelöscht)
                        sprecherText = re.sub(r'^.*\r\n', '', sprecherText)
                        sprecherText = sprecherText.strip()
                        sprecherText = re.sub(r'\r\n', '', sprecherText) # Zeilenumbrüche innerhalb des Texts
                        sprecherText = re.sub(r'\s+', ' ', sprecherText) # mehrfache Leerzeichen werden zu einem
                        
                        # write to CSV
                        # only write if name is not empty (otherwise there will be empty rows in between)
                        if(name != ""):
                            f.write(f'{num}|{name}|{sprecherText}|2\n')
                            print(f'{num}|{name}|{sprecherText}|2\n')
                    else:
                        f.write(f'{num}|Sprechertext|{sprecherText}|2 \n')
                        print(f'{num}|Sprechertext|{sprecherText}|2\n')
                
get_scripts()

Nr.|Sprecher/Regie|Text|Filmnr.

0|Sprechertext||2

2|Sprechertext|THE LORD OF THE RINGS: THE TWO TOWERS|2

4|Screenplay by|Peter Jackson, Fran Walsh and Philippa Boyens.|2

6|Based on "The Lord of The Rings" trilogy by|J.R.R Tolkien.|2

8|Sprechertext|Transcription credits|2

10|Accela, Aina, Bad burn, Bridget Chubb,|Brionn Equus (Lochrann), Drusilia, Elf Lady, %owyn Unquendor, Feanari, Finafyr, Flourish, Galadriel, Heri, Julamb, JustinsIce(Mdjasrie), Kazren, Krystal, Lady%owynKenobi, Lady Evenstar, Legolas%Bow, Lithorose, Melody, Mormegil, Nilmandra, Padfoot, Penwiper, Pilgrim Grey, Primula Baggins, Randy Savage, Samwise the Brave, Sirius Black, Tethra, The Lidless Eye, Turnar, Xyla, Yaksha|2

12|Elvish dialogue from The Elvish Linguistic|Fellowship.|2

14|Sprechertext||2

16|Regie|[TITLE: THE LORD OF THE RINGS]|2

18|Sprechertext||2

20|Regie|[Camera pans over the Misty Mountains as voices drift in from the background.]|2

22|GANDALF|You cannot pass!|2

24|FRODO|Gandalf!|2

26|GANDA

**ToDo:**
* Code für Skript 1 und 3 schreiben

**Skript 2:**
1. CSV einlesen
2. Datenbereinigung von % Zeichen (Code aus anderem Notebook kopieren) + Einzelfälle von Hand ergänzen
    * inklusive: (V.O.) und evtl. weitere
3. erste Rows mit unwichtigen Infos löschen (anhand er Indices)
4. alle Rows mit "Sprechertext" in der 2. Spalte löschen
5. Regieanweisungen:
    *  loop über alles zum Filtern: alle Rows mit Regieanweisungen rausfiltern (wenn "Regie" in 2. Spalte, dann rausfiltern)
    *  loop über gefilterten Text: suchen nach Text in [], speichern in Array
    *  loop über alles: suchen nach Text in Arrays, prüfen ob Regieanweisung vor oder nach dem Sprechertext steht
    *  daraus eine eigene Spalte machen nach dem Muster Nr.|"Regie"|Text (wenn Regieanweisung vor dem Text, Zeile davor einfügen, wenn Regieanweisung nach dem Text, Zeile danach einfügen)
6. Elbische Passagen aussortieren:
    * Suche nach Text in ()
    * überprüfen, ob es problematische Fälle gibt (Englisch - Elbisch - Übersetzung ODER: Elbisch - Übersetzung - Elbisch - Übersetzung) -> maßgeschneiderte Lösung für die Sätze, konkreten Wortlaut des Elbischen löschen
    * Regelfall: Alles vor der Übersetzung d.h. alles vor dem Text in runden Klammern löschen
7. Fortlaufende Nummer überschreiben: Row Indices verwenden